In [0]:
from pyspark.sql import functions as F

csv_path = "/Volumes/dev_flowers/bronze/files/Iris.csv"
target_table = "dev_flowers.bronze.iris"

df_raw = (
    spark.read
         .option("header", True)
         .option("inferSchema", True)
         .csv(csv_path)
)

mapping = {
    "SepalLengthCm": "sepal_length",
    "SepalWidthCm":  "sepal_width",
    "PetalLengthCm": "petal_length",
    "PetalWidthCm":  "petal_width",
    "Species":       "species",
}

df = df_raw.select([F.col(src).alias(dst) for src, dst in mapping.items()])

df = (df
      .withColumn("sepal_length", F.col("sepal_length").cast("double"))
      .withColumn("sepal_width",  F.col("sepal_width").cast("double"))
      .withColumn("petal_length", F.col("petal_length").cast("double"))
      .withColumn("petal_width",  F.col("petal_width").cast("double"))
      .withColumn("species",      F.col("species").cast("string"))
)

display(df.limit(10))

(df.write
   .format("delta")
   .mode("overwrite")
   .option("overwriteSchema", "true")
   .saveAsTable(target_table)
)
